# Overview

*ide49* is based on [Balena](https://www.balena.io/what-is-balena), a version of [Docker](https://www.docker.com/) optimized for IoT devices, rather than datacenters.

```{figure} figures/app.png
:alt: balena_app
:width: 400px
:align: center

Structure of a Docker application. A single Linux kernel is shared between the host and one or more user containers.
```

The balenaOS takes the function of the host operating system. Unlike other operating systems (Windows, MacOS, ...), it is highly specialized to run individual services in so-called containers. In *ide49* containers include micropython, code-server, duplicati, etc.

A set of [configuration files](https://github.com/iot49/ide49) describes the application. The [docker-compose.yml](https://github.com/iot49/ide49/blob/main/docker-compose.yml) file lists all user containers along with configuration parameters and information about how they interact. E.g. containers may be permitted to share files (`volumes`) or communicate over dedicated networks that are inaccessible from outside.

Each container presents a separate instance of Linux, and usually implement just one function. This arrangement minimizes undesired interaction e.g. between incompatible libraries used in various application programs. It's like having a separate computer for each task (code editor, backup, etc).

The software installed in each container is described with a Dockerfile. For example, the first line in the Dockerfile for the [micropython](https://github.com/iot49/ide49/blob/main/core/micropython/Dockerfile) specifies that this service is derived from a slightly customized version of an "official" [Jupyter docker image](https://jupyter-docker-stacks.readthedocs.io/en/latest/using/selecting.html#jupyter-scipy-notebook). This image is based on Ubuntu Linux and includes, notably, the Jupyter Notebook server as well as a host of libraries for scientific computation and plotting. Following the FROM statement are instructions for installing additional software and configuration files including the IoT Kernel used for interacting with the MicroPython REPL. 

## BalenaEngine

In Balena, the balenaEngine takes the function of the docker deamon in standard docker apps. It is accessed from the host with the `balena-engine` command. 

In [1]:
%%service host
balena-engine --help


Usage:	balena-engine [OPTIONS] COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/home/root/.balena-engine")
  -c, --context string     Name of the context to use to connect to the
                           daemon (overrides DOCKER_HOST env var and
                           default context set with "docker context use")
  -D, --debug              Enable debug mode
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/home/root/.balena-engine/ca.pem")
      --tlscert string     Path to TLS certificate file (default
                           "/home/root/.balena-e

To get a list of all running containers, type:

In [2]:
%%service host
balena-engine ps --format "table {{.Names}}\t{{.Size}}\t{{.Image}}"

NAMES                         SIZE                      IMAGE
jupyter_3834077_1864199       27.9kB (virtual 1.7GB)    17ed674e39be
esp-idf_3834084_1864199       27.9kB (virtual 2.25GB)   5ccaf3068e5a
balena-cli_3834085_1864199    0B (virtual 1.62GB)       74a76eccf611
arm32_3834083_1864199         0B (virtual 1.69GB)       5a7c64cb8d95
nginx_3834076_1864199         3B (virtual 28.9MB)       0fc36cfd415e
duplicati_3834079_1864199     333kB (virtual 659MB)     70aa02f766b5
code-server_3834078_1864199   334kB (virtual 543MB)     eab6f75bba26
plotserver_3834082_1864199    0B (virtual 590MB)        f3ee00d5c1c6
mosquitto_3834081_1864199     0B (virtual 9.95MB)       05117273e504
smb_3834080_1864199           355kB (virtual 225MB)     edaa3074ac12
resin_supervisor              45B (virtual 66.3MB)      registry2.balena-cloud.com/v2/d889d3981987f55bc74a78989588bf14:latest


The balenaEngine has many more features. For example, to get the build history for the arm32 image run

In [1]:
%%service host
balena-engine history 5a7c64cb8d95 --format "table {{.Size}}\t{{.CreatedBy}}" # --no-trunc

SIZE                CREATED BY
0B                  /bin/sh -c #(nop)  CMD ["/bin/bash" "/usr/lo…
0B                  /bin/sh -c #(nop) WORKDIR /home/iot
0B                  /bin/sh -c #(nop)  USER iot
336kB               /bin/sh -c chmod a+rx /usr/local/bin/start.s…
699B                /bin/sh -c #(nop) COPY file:c2e922284a361f02…
826MB               /bin/sh -c install_packages     git     cifs…
0B                  /bin/sh -c #(nop)  ENV UDEV=1
122kB               /bin/sh -c echo '#!/bin/sh.real\nbalena-info…
616B                /bin/sh -c [ ! -d /.balena/messages ] && mkd…
1.32MB              /bin/sh -c curl -SLO "https://raw.githubuser…
0B                  /bin/sh -c #(nop)  CMD ["echo" "'No CMD comm…
0B                  /bin/sh -c #(nop)  ENV PYTHONPATH=/usr/lib/p…
50B                 /bin/sh -c cd /usr/local/bin  && ln -sf pip3…
1.78MB              /bin/sh -c set -x  && mkdir -p /usr/src/dbus…
43MB                /bin/sh -c apt-get update && apt-get install…
0B                  /bi

## Networking

Containers use networks for communication. *ide49* makes use of two separate networks:

* an internal network used only for communication within the app, and
* the host network, used to access the Internet.

Most containers have access only to the internal network. The `nginx` webserver acts as a reverse proxy to pass requests received from browsers on port 443 (and port 80, which is forwarded to 443) to the appropriate container on the internal network. The single ingress allows centralized handling of encryption and password verification in one place. The nginx configration is at

In [2]:
!cat /service-config/nginx/nginx.conf

# /etc/nginx/nginx.conf

user                                nginx;
worker_processes                    1;

error_log                           /var/log/nginx/error.log warn;
pid                                 /var/run/nginx.pid;


events {
    worker_connections              256;
}

http {
    include                         /etc/nginx/mime.types;
    default_type                    application/octet-stream;

    log_format main                 '$remote_addr - $remote_user "$request" '
                                    '$status $body_bytes_sent "$http_referer" '
                                    '"$http_x_forwarded_for"';

    access_log                      /var/log/nginx/access.log main;

    sendfile                        on;
    keepalive_timeout               65;
    # gzip                          on;

    # http -> https redirect
    server {
        listen                      80;
        return                      301 https://$host$request_uri;
    }

    server {

   

The network confiuration is available from the balena-engine:

In [1]:
%%service host
balena-engine ps --format "table {{.Names}}\t{{.Ports}}"

NAMES                         PORTS
jupyter_3834077_1864199       
esp-idf_3834084_1864199       
balena-cli_3834085_1864199    8889/tcp
arm32_3834083_1864199         
nginx_3834076_1864199         0.0.0.0:80->80/tcp, 0.0.0.0:443->443/tcp
duplicati_3834079_1864199     8200/tcp
code-server_3834078_1864199   8443/tcp
plotserver_3834082_1864199    8080/tcp
mosquitto_3834081_1864199     0.0.0.0:1883->1883/tcp
smb_3834080_1864199           0.0.0.0:139->139/tcp, 0.0.0.0:445->445/tcp
resin_supervisor              


## Storage

Broadly, [Docker](https://docs.docker.com/storage/) containers distinguish two kinds of data storage.

### Owned by the container

That's the default and includes all operating system related files, installed software (compilers, etc). 

Although it's possible to make changes (e.g. with `apt-get` run inside the container), those changes do not persist between updates and hence should be avoided. Best practice is not to modify files "owned" by the container. 

This part of storage is "private" to the container and invisible from other containers in the same app.

### Mounted into the container (volumes)

Storage can also be "mounted" from the "host", the underlying operating system that runs the Docker app, *ide49*. For example, `/home/iot` and the subfolders of `/service-config` are mounted. The `volumes` section of the `docker-compose.yml` file lists all mounts.

The same volumes can be mounted into several containers (at the same or different locations). This enables data sharing. For example, all volumes are mounted in the `duplicati` container for backup and `micropython` and `code-server` containers for editing. 

For example the configuration for `nginx` (webserver), is mounted in `/service-config` in the `code-server` and `micropython` containers for editing, and at `/etc/nginx` in the `nginx` service.

Changes to volumes persist between container updates. 

### Samba fileshare

In addition to the types of storage discussed above, *ide49* can also be configured to mount `/home/iot` from a Samba file server.

Uses include sharing the same data between several *ide49* instances or in situations where a shared file server, perhaps with centralized backup, is preferred.

For convenience, the entire home directory (`/home/iot`) is mounted. This can cause some undesired behavior. For example, databases frequently use status files (e.g. locks) used by the running instance. Sharing those with another device (running perhaps a different instance of the same database) results in conflicts.

To prevent this, the local iot volume is always available at `/service-config/iot-home`. Unlike `/home/iot`, this copy is never "hidden" by a Samba mount. *ide49* is configured to store status information (e.g. for `jupyter`) in `/service-config/iot-home` rather than `/home/iot`.

### Mounts

The `duplicati` service automtically mounts USB storage devices at `/mnt`. These devices can also be manually mounted in the `jupyter` container following standard Linux practice. Avoid simultaneous access of the same storage device from multiple containers to avoid data corruption.

Adapt the sample script to your situation.

In [ ]:
%%bash

# 1) find device, e.g. /dev/sdc1
sudo fdisk -l

# 2) create mount point
sudo mkdir -p /mnt/media

# 3) mount volume
sudo mount /dev/sdc1 /mnt/media

# 4) use data ...
ls /mnt/media

# 5) unmount
sudo umount /mnt/media